#Adam Kritz Crate and Barrel Project

Welcome to my second project for data mining, the crate and barrel project. Today I will be classifying products into their proper categories for the Otto Group. Now in real life I may be a far left anarcho-socialist, but for today only I will bend the knee to large coporations, and do the bidding of the Otto Group. 

Contents:

1. Get those files in
2. Explore the data
3. Split the data
4. Bagging and Pasting
5. Random Subspace and Random Patches
6. XGBoost
7. Neural Network
8. Final Training
9. Conclusion


#Read the files

To start, I will upload the files and read them in. The following commands will only work properly if you have the same files, the training and testing data from the Kaggle website for the Otto Group project. The files must also be the same name as mine, "kaggletrain.csv" and "kaggletest.csv". Lastly, they must be in the same place as my files, uploaded alongside the "sample_data". 

In [1]:
!ls

kaggletest.csv	kaggletrain.csv  sample_data


In [2]:
import pandas as pd
kaggletrain = pd.read_csv('kaggletrain.csv', index_col = 'id')
kaggletest = pd.read_csv('kaggletest.csv', index_col = 'id')

I went ahead and imported the ID as the index to save myself some trouble later.

#Explore the data

Now it is time to see what this data is all about. I already know from the directions that I will be classifying the data categorically. The features are all in numerical data, with counts representing the occurences of different events. I also know there are 93 features to look at and more than 200,000 products. Now to take a look at the data.

In [3]:
kaggletrain.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,1,0,4,1,1,0,0,2,0,0,0,0,0,1,0,0,0,0,1,...,0,0,2,0,0,11,0,1,1,0,1,0,7,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,6,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
4,1,0,0,1,6,1,5,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,7,2,2,0,0,0,58,0,10,0,0,0,0,0,3,0,0,...,1,0,0,0,0,0,0,0,0,0,2,1,5,0,0,4,0,0,2,1,0,1,0,0,1,1,2,2,0,22,0,1,2,0,0,0,0,0,0,Class_1
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,3,0,0,0,0,0,0,0,4,0,1,0,0,0,1,0,0,0,0,1,0,0,0,Class_1


In [4]:
len(kaggletrain)

61878

In [5]:
kaggletest.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,3,0,0,0,3,2,1,0,0,0,0,0,0,0,5,3,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,11,1,20,0,0,0,0,0
2,2,2,14,16,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,2,0,4,0,4,0,0,0,0,2,0,0,0,8,0,0,0,0,0,...,24,0,0,0,0,0,0,0,0,0,6,8,0,0,0,0,0,0,0,0,0,0,0,2,0,0,4,0,2,0,0,0,0,0,0,4,0,0,2,0
3,0,1,12,1,0,0,0,0,0,0,7,1,0,0,0,7,0,2,0,0,0,4,0,0,0,1,1,2,0,0,0,0,0,0,1,0,0,2,0,0,...,10,1,0,2,0,0,1,6,1,1,0,0,1,1,1,2,0,0,2,0,0,0,0,0,0,0,6,0,2,0,0,0,0,0,2,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,3,0,0,0,0,0,0,0,0,4,0,0,0,1,0,0,0,2,0,0,0,0,1,0,0,...,2,0,0,0,0,0,0,0,9,0,0,2,0,0,0,0,6,0,8,0,0,0,0,1,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0
5,1,0,0,1,0,0,1,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,4,0,5,16,0,0,0,0,0,0,1,0,0,0,0,0,0,0,9,0,0


In [6]:
len(kaggletest)

144368

Already off the bat there is some interesting stuff to note. The training set is actually far smaller than the testing set, which I have not dealt with yet. This may affect how accurate certain classifiers are. On top of this, the test set does not actually have the true labels for the data in it. This was likely to prevent cheating in the competition. 

Now to take a look at the description of the data.

In [7]:
kaggletrain.describe(include="all")

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
count,61878.00000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,...,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Class_2
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16122
mean,0.38668,0.263066,0.901467,0.779081,0.071043,0.025696,0.193704,0.662433,1.011296,0.263906,1.252869,0.140874,0.480979,1.696693,1.284398,1.413459,0.366108,0.575423,0.551699,0.471525,0.204014,0.729969,0.142522,2.643880,1.534520,0.563108,0.696613,0.238970,0.275768,0.150312,0.148680,1.043796,0.696516,0.946411,0.666263,0.709089,0.263632,0.582129,0.485585,1.653059,...,0.323120,0.303775,0.309108,0.697970,0.388603,1.029930,0.239746,1.187563,0.168590,1.256796,0.222228,0.571706,2.897653,0.392902,0.811128,0.892789,0.319290,0.858722,0.591050,0.579851,0.726817,0.748457,0.124196,0.366415,0.300446,0.698067,0.078461,0.187983,0.496719,0.070752,0.532306,1.128576,0.393549,0.874915,0.457772,0.812421,0.264941,0.380119,0.126135,NaN
std,1.52533,1.252073,2.934818,2.788005,0.438902,0.215333,1.030102,2.255770,3.474822,1.083340,3.042333,0.567089,2.014697,3.163212,3.862236,2.226163,1.477436,1.335985,4.636145,1.438727,0.696050,1.446220,0.782979,4.629015,2.332994,1.710305,2.873222,0.828112,1.901294,1.640880,0.897354,2.416849,1.310202,3.368622,3.197965,2.555119,0.756934,1.602579,3.298315,3.299798,...,0.998743,1.925806,1.082148,3.983722,2.577693,3.028469,1.017553,2.666742,0.946158,3.402080,0.783052,1.361874,4.974322,1.761054,4.111091,1.941368,1.162443,2.411646,5.783233,3.757822,3.200095,2.920038,0.906621,2.778317,1.285569,2.245671,0.46124

In [8]:
kaggletest.describe(include="all")

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
count,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,...,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000,144368.000000
mean,0.386201,0.263597,0.899819,0.780727,0.071498,0.026439,0.200169,0.667378,1.035271,0.268778,1.253962,0.143716,0.486361,1.699566,1.290411,1.428017,0.366168,0.582387,0.560671,0.484256,0.202649,0.738640,0.140724,2.658906,1.540404,0.562666,0.690035,0.240663,0.277229,0.149687,0.159190,1.051847,0.705087,0.945009,0.680518,0.719855,0.270718,0.595596,0.486174,1.638320,...,2.082740,0.328085,0.303274,0.312742,0.691365,0.385362,1.020295,0.247347,1.188587,0.174519,1.272422,0.231942,0.576887,2.914517,0.369292,0.828023,0.907403,0.333045,0.859574,0.604961,0.601809,0.710871,0.768190,0.125298,0.379004,0.312438,0.713295,0.076852,0.191892,0.520489,0.074989,0.538485,1.128782,0.405249,0.875526,0.473284,0.814010,0.271161,0.388348,0.132675
std,1.468882,1.261908,2.949106,2.846181,0.428568,0.228354,1.069235,2.286832,3.548618,1.187347,3.009389,0.568097,2.032072,3.159962,3.869359,2.242769,1.453056,1.335440,4.760191,1.462113,0.709394,1.456484,0.754854,4.625318,2.339625,1.764489,2.781712,0.829157,1.941813,1.620535,1.071093,2.452233,1.318721,3.414385,3.247933,2.666145,0.792202,1.607475,3.314148,3.268989,...,4.147678,0.995321,1.904421,1.118638,4.037591,2.478530,2.982277,1.053898,2.711372,1.064165,3.428788,0.864261,1.395459,5.034601,1.650509,4.144378,1.941207,1.254570,2.402588,5.979542,3.954618,3.076905,3.008035,0.907691,2.837345,1.330912,2.326912,0.455660,0.839957,2.597924,1.288595,1.906121,2.682511,1.631566,2.090288,1.617853,4.603653,2.073627,1.006935,1.302695
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,

It seems like for both data sets the most common value is 0, with some higher values mixed in every once in a while that bring up the mean. I will test this theory by looking at the value counts of one of the columns in the training set.

In [9]:
kaggletrain['feat_1'].value_counts()

0     51483
1      5906
2      1829
3       981
4       521
5       471
6       207
7       192
8        71
9        55
22       32
11       24
10       15
13       10
15        8
26        6
19        5
24        5
12        5
28        4
14        4
21        4
23        4
17        4
48        3
16        3
27        3
25        3
39        2
31        2
42        2
43        2
30        2
47        2
56        1
20        1
37        1
34        1
29        1
61        1
32        1
40        1
Name: feat_1, dtype: int64

It seems like my prediction was relatively correct, higher numbers of events occur far less commonly than lower numbers.

Lastly I will just double check that both data sets do not contain null values that could mess up classifiers. 

In [10]:
kaggletrain.isnull().values.any()

False

In [11]:
kaggletest.isnull().values.any()

False

Excellent!

#Split the Data

Instead of using the test data to check accuracy, I will just split the training data into new testing and training groups, as I cannot repeatedly submit to kaggle to check accuracy. In this case I will not control randomness so I can test my shorter classifiers on multiple random sets.

In [12]:
ktrainlabels = kaggletrain['target']
ktrainfeat = kaggletrain.drop('target', axis = 1)
from sklearn.model_selection import train_test_split

In [13]:
ktrainf, ktestf, ktrainl, ktestl = train_test_split(ktrainfeat, ktrainlabels, test_size = 0.2)

Everything seems to be in order, now onto training.

#Bagging and Pasting 

I will start by trying out Bagging and Pasting on both decision tree and KNN classifiers.

In [14]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score


clf = tree.DecisionTreeClassifier(criterion='entropy')
bagging_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
bagging_clf.fit(ktrainf, ktrainl)
predictions = bagging_clf.predict(ktestf)
accuracy_score(ktestl, predictions)

0.622737556561086

In [15]:
clf = tree.DecisionTreeClassifier(criterion='entropy')
pasting_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=False, n_jobs=-1)
pasting_clf.fit(ktrainf, ktrainl)
predictions = bagging_clf.predict(ktestf)
accuracy_score(ktestl, predictions)

0.622737556561086

In [16]:
knn = KNeighborsClassifier(n_neighbors=3)
bagging_knn = BaggingClassifier(knn, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
bagging_knn.fit(ktrainf, ktrainl)
predictions = bagging_clf.predict(ktestf)
accuracy_score(ktestl, predictions)

0.622737556561086

In [17]:
knn = tree.DecisionTreeClassifier(criterion='entropy')
pasting_knn = BaggingClassifier(knn, n_estimators=20, max_samples=100, 
                                bootstrap=False, n_jobs=-1)
pasting_knn.fit(ktrainf, ktrainl)
predictions = bagging_clf.predict(ktestf)
accuracy_score(ktestl, predictions)

0.622737556561086

This is a pretty atrocious start after a couple of runs, I will come back to these algorithms later to fix them up if the others are also not performing well.

#Random Subspace and Random Patches

Next I will try random subspace and patches again on decision tree and knn classifiers.

In [18]:
clf = tree.DecisionTreeClassifier(criterion='entropy')
subspace_clf = BaggingClassifier(clf, n_estimators=50, max_features=7, 
                                bootstrap=True, n_jobs=-1)
subspace_clf.fit(ktrainf, ktrainl)
predictions = bagging_clf.predict(ktestf)
accuracy_score(ktestl, predictions)

0.622737556561086

In [19]:
clf = tree.DecisionTreeClassifier(criterion='entropy')
patches_clf = BaggingClassifier(clf, n_estimators=100, max_features=7, 
                                 max_samples=100, bootstrap=False, n_jobs=-1)
patches_clf.fit(ktrainf, ktrainl)
predictions = bagging_clf.predict(ktestf)
accuracy_score(ktestl, predictions)

0.622737556561086

In [20]:
knn = tree.DecisionTreeClassifier(criterion='entropy')
subspace_knn = BaggingClassifier(knn, n_estimators=50, max_features=7, 
                                bootstrap=True, n_jobs=-1)
subspace_knn.fit(ktrainf, ktrainl)
predictions = bagging_clf.predict(ktestf)
accuracy_score(ktestl, predictions)

0.622737556561086

In [21]:
knn = tree.DecisionTreeClassifier(criterion='entropy')
patches_knn = BaggingClassifier(knn, n_estimators=100, max_features=7, 
                                 max_samples=100, bootstrap=False, n_jobs=-1)
patches_knn.fit(ktrainf, ktrainl)
predictions = bagging_clf.predict(ktestf)
accuracy_score(ktestl, predictions)

0.622737556561086

Again these classifiers are not performing the best, so I will come back to them later if the others fail.

#XGBoost

Time to bring out the big guns, I will try an XGBoost classifier.

In [22]:
from xgboost import XGBClassifier
params = {"n_estimators": 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}
Boost = XGBClassifier(**params)

Boost.fit(ktrainf, ktrainl)
predictions = Boost.predict(ktestf)
accuracy_score(ktestl, predictions)

0.7954104718810601

Holy moly, 79% accuracy right off the back, that blows the other ones out of the water. Let me see if I can improve anymore. I should note I originally tried using a gridsearch for this, but it was far too slow, even with minimal parameters, so I changed my mind.

Changing the max depth. This seemed to help a lot.

In [23]:
params = {"n_estimators": 400, "max_depth": 12, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}
Boost = XGBClassifier(**params)

Boost.fit(ktrainf, ktrainl)
predictions = Boost.predict(ktestf)
accuracy_score(ktestl, predictions)

0.8265190691661279


Adjusting the learning rate seemed beneficial in some cases, but not in others, so I just left it as the default.


In [24]:
params = {"n_estimators": 400, "max_depth": 12, 'learning_rate': 0.3, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}
Boost = XGBClassifier(**params)

Boost.fit(ktrainf, ktrainl)
predictions = Boost.predict(ktestf)
accuracy_score(ktestl, predictions)

0.8257110536522301

Adjusting the subsample to prevent overfitting. This did not seem to help.

In [25]:
params = {"n_estimators": 400, "max_depth": 12, 'learning_rate': 0.3, 'subsample': 0.5, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}
Boost = XGBClassifier(**params)

Boost.fit(ktrainf, ktrainl)
predictions = Boost.predict(ktestf)
accuracy_score(ktestl, predictions)

0.8253070458952811

I will go with the standard XGBoost with a max_depth of 12.

#Neural Network

After an impressive showing by XGBoost, I shall see how a neural network performs. 

Before creating a neural network, I need to do some work on the labels to get them in the right format.

In [55]:
ktrainl.value_counts()

Class_2    12924
Class_6    11308
Class_8     6770
Class_3     6357
Class_9     3978
Class_7     2276
Class_5     2170
Class_4     2166
Class_1     1553
Name: target, dtype: int64

In [66]:
k_trainl = ktrainl.replace({'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3, 'Class_5':4,
                            'Class_6': 5, 'Class_7': 6, 'Class_8': 7, 'Class_9': 8})
k_testl = ktestl.replace({'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3, 'Class_5':4,
                            'Class_6': 5, 'Class_7': 6, 'Class_8': 7, 'Class_9': 8})
k_trainl

id
52206    7
26746    3
7562     1
8110     1
25041    2
        ..
7161     1
13239    1
7887     1
47564    6
42964    5
Name: target, Length: 49502, dtype: int64

In [69]:
from keras.utils import to_categorical

ktrain_labels = to_categorical(k_trainl, num_classes=9)
ktest_labels = to_categorical(k_testl, num_classes=9)

ktrain_labels 

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [70]:
ktrain_labels[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)

Now for the neural network.

In [72]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(500, activation='relu'))
network.add(layers.Dense(9, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(ktrainf, ktrain_labels, epochs=15, batch_size=100)

Epoch 1/15
496/496 [==============================] - 1s 2ms/step - loss: 0.8257 - accuracy: 0.7216
Epoch 2/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5768 - accuracy: 0.7857
Epoch 3/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5270 - accuracy: 0.7989
Epoch 4/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4893 - accuracy: 0.8129
Epoch 5/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4666 - accuracy: 0.8220
Epoch 6/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4522 - accuracy: 0.8283
Epoch 7/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4310 - accuracy: 0.8343
Epoch 8/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4098 - accuracy: 0.8430
Epoch 9/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4019 - accuracy: 0.8446
Epoch 10/15
496/496 [==============================] - 1s 2ms/step - loss: 0.3771 - accuracy: 0.8548

In [73]:
kaggle_loss, kaggle_acc = network.evaluate(ktestf, ktest_labels)
print('Accuracy:', kaggle_acc)

387/387 [==============================] - 1s 2ms/step - loss: 0.5974 - accuracy: 0.8046
Accuracy: 0.8046218752861023


This a pretty great start for the neural network, I will see if I can change up the hyper paramters to make it even better. 

For starters, I'll add another layer.

In [74]:
network = models.Sequential()
network.add(layers.Dense(500, activation='relu'))
network.add(layers.Dense(200, activation='sigmoid'))
network.add(layers.Dense(9, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(ktrainf, ktrain_labels, epochs=15, batch_size=100)

Epoch 1/15
496/496 [==============================] - 2s 2ms/step - loss: 0.7718 - accuracy: 0.7226
Epoch 2/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5427 - accuracy: 0.7849
Epoch 3/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5002 - accuracy: 0.8006
Epoch 4/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4707 - accuracy: 0.8133
Epoch 5/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4513 - accuracy: 0.8197
Epoch 6/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4262 - accuracy: 0.8293
Epoch 7/15
496/496 [==============================] - 1s 2ms/step - loss: 0.4073 - accuracy: 0.8367
Epoch 8/15
496/496 [==============================] - 1s 3ms/step - loss: 0.3861 - accuracy: 0.8445
Epoch 9/15
496/496 [==============================] - 1s 2ms/step - loss: 0.3656 - accuracy: 0.8564
Epoch 10/15
496/496 [==============================] - 1s 2ms/step - loss: 0.3467 - accuracy: 0.8615

In [75]:
kaggle_loss, kaggle_acc = network.evaluate(ktestf, ktest_labels)
print('Accuracy:', kaggle_acc)

387/387 [==============================] - 1s 2ms/step - loss: 0.5494 - accuracy: 0.8087
Accuracy: 0.808661937713623


Not much better.

Now I will change up the batch size.

In [76]:
network = models.Sequential()
network.add(layers.Dense(500, activation='relu'))
network.add(layers.Dense(200, activation='sigmoid'))
network.add(layers.Dense(9, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(ktrainf, ktrain_labels, epochs=15, batch_size=20)

Epoch 1/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.7032 - accuracy: 0.7402
Epoch 2/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.5424 - accuracy: 0.7922
Epoch 3/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.5158 - accuracy: 0.8008
Epoch 4/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.4834 - accuracy: 0.8152
Epoch 5/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.4719 - accuracy: 0.8192
Epoch 6/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.4512 - accuracy: 0.8274
Epoch 7/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.4455 - accuracy: 0.8317
Epoch 8/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.4251 - accuracy: 0.8391
Epoch 9/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.4200 - accuracy: 0.8404
Epoch 10/15
2476/2476 [==============================] - 6s 2ms/step - loss: 0.406

In [77]:
kaggle_loss, kaggle_acc = network.evaluate(ktestf, ktest_labels)
print('Accuracy:', kaggle_acc)

387/387 [==============================] - 1s 2ms/step - loss: 0.6143 - accuracy: 0.8013
Accuracy: 0.8013089895248413


I am going the wrong direction on my accuracy.

How about changing some activation conditions?

In [78]:
network = models.Sequential()
network.add(layers.Dense(500, activation='swish'))
network.add(layers.Dense(200, activation='tanh'))
network.add(layers.Dense(9, activation='gelu'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(ktrainf, ktrain_labels, epochs=15, batch_size=100)

Epoch 1/15
496/496 [==============================] - 2s 2ms/step - loss: 2.4249 - accuracy: 0.6415
Epoch 2/15
496/496 [==============================] - 1s 2ms/step - loss: 1.6428 - accuracy: 0.6985
Epoch 3/15
496/496 [==============================] - 1s 2ms/step - loss: 2.6786 - accuracy: 0.6619
Epoch 4/15
496/496 [==============================] - 1s 2ms/step - loss: 3.1706 - accuracy: 0.6291
Epoch 5/15
496/496 [==============================] - 1s 2ms/step - loss: 4.6459 - accuracy: 0.5683
Epoch 6/15
496/496 [==============================] - 1s 2ms/step - loss: 3.3419 - accuracy: 0.6358
Epoch 7/15
496/496 [==============================] - 1s 2ms/step - loss: 3.8837 - accuracy: 0.6060
Epoch 8/15
496/496 [==============================] - 1s 2ms/step - loss: 4.2186 - accuracy: 0.5879
Epoch 9/15
496/496 [==============================] - 1s 2ms/step - loss: 3.6830 - accuracy: 0.6508
Epoch 10/15
496/496 [==============================] - 1s 2ms/step - loss: 3.4966 - accuracy: 0.6502

In [79]:
kaggle_loss, kaggle_acc = network.evaluate(ktestf, ktest_labels)
print('Accuracy:', kaggle_acc)

387/387 [==============================] - 1s 2ms/step - loss: 5.3248 - accuracy: 0.6196
Accuracy: 0.6195862889289856


Never mind that sucked.

Let me try to make a few more changes.

In [80]:
network = models.Sequential()
network.add(layers.Dense(500, activation='relu'))
network.add(layers.Dense(200, activation='sigmoid'))
network.add(layers.Dense(9, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(ktrainf, ktrain_labels, epochs=50, batch_size=100)

Epoch 1/50
496/496 [==============================] - 2s 2ms/step - loss: 0.7901 - accuracy: 0.7162
Epoch 2/50
496/496 [==============================] - 1s 2ms/step - loss: 0.5449 - accuracy: 0.7849
Epoch 3/50
496/496 [==============================] - 1s 2ms/step - loss: 0.5033 - accuracy: 0.8033
Epoch 4/50
496/496 [==============================] - 1s 2ms/step - loss: 0.4758 - accuracy: 0.8112
Epoch 5/50
496/496 [==============================] - 1s 2ms/step - loss: 0.4541 - accuracy: 0.8198
Epoch 6/50
496/496 [==============================] - 1s 2ms/step - loss: 0.4238 - accuracy: 0.8314
Epoch 7/50
496/496 [==============================] - 1s 2ms/step - loss: 0.4110 - accuracy: 0.8349
Epoch 8/50
496/496 [==============================] - 1s 3ms/step - loss: 0.3847 - accuracy: 0.8471
Epoch 9/50
496/496 [==============================] - 1s 2ms/step - loss: 0.3721 - accuracy: 0.8494
Epoch 10/50
496/496 [==============================] - 1s 2ms/step - loss: 0.3540 - accuracy: 0.8557

In [81]:
kaggle_loss, kaggle_acc = network.evaluate(ktestf, ktest_labels)
print('Accuracy:', kaggle_acc)

387/387 [==============================] - 1s 2ms/step - loss: 1.0541 - accuracy: 0.7962
Accuracy: 0.7962185144424438


In [82]:
network = models.Sequential()
network.add(layers.Dense(500, activation='sigmoid'))
network.add(layers.Dense(9, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(ktrainf, ktrain_labels, epochs=15, batch_size=50)

Epoch 1/15
991/991 [==============================] - 2s 2ms/step - loss: 0.8013 - accuracy: 0.7171
Epoch 2/15
991/991 [==============================] - 2s 2ms/step - loss: 0.5924 - accuracy: 0.7744
Epoch 3/15
991/991 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7844
Epoch 4/15
991/991 [==============================] - 2s 2ms/step - loss: 0.5428 - accuracy: 0.7898
Epoch 5/15
991/991 [==============================] - 2s 2ms/step - loss: 0.5268 - accuracy: 0.7957
Epoch 6/15
991/991 [==============================] - 2s 2ms/step - loss: 0.5149 - accuracy: 0.8003
Epoch 7/15
991/991 [==============================] - 2s 2ms/step - loss: 0.5007 - accuracy: 0.8048
Epoch 8/15
991/991 [==============================] - 2s 2ms/step - loss: 0.4932 - accuracy: 0.8099
Epoch 9/15
991/991 [==============================] - 2s 2ms/step - loss: 0.4821 - accuracy: 0.8129
Epoch 10/15
991/991 [==============================] - 2s 2ms/step - loss: 0.4730 - accuracy: 0.8156

In [83]:
kaggle_loss, kaggle_acc = network.evaluate(ktestf, ktest_labels)
print('Accuracy:', kaggle_acc)

387/387 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7868
Accuracy: 0.7867646813392639


In [84]:
network = models.Sequential()
network.add(layers.Dense(20, activation='relu'))
network.add(layers.Dense(20, activation='sigmoid'))
network.add(layers.Dense(9, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(ktrainf, ktrain_labels, epochs=15, batch_size=100)

Epoch 1/15
496/496 [==============================] - 2s 2ms/step - loss: 1.4210 - accuracy: 0.5508
Epoch 2/15
496/496 [==============================] - 1s 2ms/step - loss: 0.7174 - accuracy: 0.7400
Epoch 3/15
496/496 [==============================] - 1s 2ms/step - loss: 0.6332 - accuracy: 0.7591
Epoch 4/15
496/496 [==============================] - 1s 2ms/step - loss: 0.6136 - accuracy: 0.7660
Epoch 5/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5942 - accuracy: 0.7738
Epoch 6/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5845 - accuracy: 0.7758
Epoch 7/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5735 - accuracy: 0.7821
Epoch 8/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5741 - accuracy: 0.7792
Epoch 9/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5675 - accuracy: 0.7839
Epoch 10/15
496/496 [==============================] - 1s 2ms/step - loss: 0.5558 - accuracy: 0.7877

In [85]:
kaggle_loss, kaggle_acc = network.evaluate(ktestf, ktest_labels)
print('Accuracy:', kaggle_acc)

387/387 [==============================] - 1s 2ms/step - loss: 0.5734 - accuracy: 0.7845
Accuracy: 0.7845022678375244


I will use all the information I have here to come up with the best hyperparameters for the final training. 

#Final Training

Since I get two submissions to Kaggle, I will try both XGBoost and a Neural Network, as they both performed well. I will use the best performing hyper parameters from the training and and apply those to the full training data before I officially submit it to Kaggle.

In [113]:
params = { "n_estimators": 400, 'max_depth': 12, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor' }
Boost = XGBClassifier(**params)

Boost.fit(ktrainfeat, ktrainlabels)
Boostpredictions = Boost.predict(kaggletest)
Boostpredictions

array(['Class_4', 'Class_8', 'Class_6', ..., 'Class_2', 'Class_4',
       'Class_3'], dtype=object)

In [114]:
boost = pd.Series(Boostpredictions)
boost

0         Class_4
1         Class_8
2         Class_6
3         Class_2
4         Class_9
           ...   
144363    Class_1
144364    Class_3
144365    Class_2
144366    Class_4
144367    Class_3
Length: 144368, dtype: object

In [116]:
boost2 = boost.replace({'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3, 'Class_5':4,
                                   'Class_6': 5, 'Class_7': 6, 'Class_8': 7, 'Class_9':8})
boost3 = to_categorical(boost2)
boost3[0:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [117]:
boost4= pd.DataFrame(boost3)
boost4.head()

,0,1,2,3,4,5,6,7,8
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [118]:
boost4.index += 1 
boost4.index.name = 'id'
boost4.columns=['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']
boost4.head()

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
id,,,,,,,,,
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [86]:
ktl = ktrainlabels.replace({'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3, 'Class_5':4,
                            'Class_6': 5, 'Class_7': 6, 'Class_8': 7, 'Class_9':8})
ktl = to_categorical(ktl)
ktl

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [88]:
network = models.Sequential()
network.add(layers.Dense(500, activation='relu'))
network.add(layers.Dense(200, activation='sigmoid'))
network.add(layers.Dense(9, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(ktrainfeat, ktl, epochs=15, batch_size=100)

Epoch 1/15
619/619 [==============================] - 2s 2ms/step - loss: 0.7540 - accuracy: 0.7284
Epoch 2/15
619/619 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7908
Epoch 3/15
619/619 [==============================] - 2s 2ms/step - loss: 0.4910 - accuracy: 0.8054
Epoch 4/15
619/619 [==============================] - 1s 2ms/step - loss: 0.4667 - accuracy: 0.8135
Epoch 5/15
619/619 [==============================] - 1s 2ms/step - loss: 0.4422 - accuracy: 0.8242
Epoch 6/15
619/619 [==============================] - 1s 2ms/step - loss: 0.4238 - accuracy: 0.8301
Epoch 7/15
619/619 [==============================] - 1s 2ms/step - loss: 0.3997 - accuracy: 0.8419
Epoch 8/15
619/619 [==============================] - 2s 2ms/step - loss: 0.3839 - accuracy: 0.8478
Epoch 9/15
619/619 [==============================] - 1s 2ms/step - loss: 0.3665 - accuracy: 0.8534
Epoch 10/15
619/619 [==============================] - 1s 2ms/step - loss: 0.3494 - accuracy: 0.8614

In [89]:
predictions = network.predict(kaggletest)
net = pd.DataFrame(predictions)
net.index += 1 
net.index.name = 'id'
net.columns=['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']

In [90]:
net

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
id,,,,,,,,,
1,3.696331e-05,7.341960e-01,1.714459e-01,8.921132e-02,5.041884e-07,3.538568e-06,5.104412e-03,3.225402e-07,1.050146e-06
2,5.933168e-05,5.199404e-05,1.483165e-05,4.496659e-07,1.121173e-07,9.022815e-01,6.828240e-05,9.721024e-02,3.131813e-04
3,2.366898e-11,1.438781e-08,7.401856e-10,1.169440e-08,1.399982e-10,9.999964e-01,3.502934e-08,3.404164e-06,1.421668e-07
4,1.793807e-07,8.985019e-01,1.002394e-01,1.249216e-03,1.376274e-08,7.718728e-09,6.977607e-07,2.413338e-07,8.204160e-06
5,6.223351e-02,3.559636e-06,1.493112e-07,2.525925e-08,4.666263e-10,1.622618e-03,1.144699e-05,2.575807e-03,9.335529e-01
...,...,...,...,...,...,...,...,...,...
144364,6.271170e-03,4.441265e-06,1.934859e-07,2.559826e-05,7.316202e-08,9.891137e-01,1.852310e-04,3.840471e-03,5.590571e-04
144365,8.932214e-04,2.163854e-01,5.454647e-01,4.796935e-02,3.566733e-06,8.339478e-05,1.891906e-01,1.839120e-06,8.079296e-06
144366,7.517587e-11,6.052749e-01,3.803245e-01,1.292199e-02,8.318403e-07,1.364874e-05,1.461804e-03,6.141050e-07,1.635235e-06


In [119]:
boost4.to_csv('boostresults.csv')

In [120]:
net.to_csv('networkresults.csv')

#Conclusion



**Results**

XGBoost: public score = 5.89733 ||||| private score = 5.91635

Neural Network: public score = 0.54523 ||||| private score = 0.54644

**The images of my results are in the projects folder of my github repository.**

So my results were not exactly what I expected. My XGBoost classifier, which I thought would perform better, totally bombed. However, my neural network classifier performed quite well. I am not sure if I made a mistake somewhere, or if XGBoost really just did not work well. Regardless I am happy with my neural network score, 0.54 seems quite good!